In [8]:
#!pip install pyreadstat
import pyreadstat
import pandas as pd

df_path = "anes_timeseries_2020_stata_20210324.dta"


[ 1. -1.  2. -9. -8.]


In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
